In [8]:
import numpy as np
#from skimage.io import imread
import matplotlib.pyplot as plt
from tensorflow.keras.applications.imagenet_utils import decode_predictions
import tensorflow as tf
import os

from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers.experimental import preprocessing

from tensorflow.keras.applications.efficientnet import EfficientNetB0, EfficientNetB7, EfficientNetB1, EfficientNetB2, EfficientNetB3
#from efficientnet import EfficientNetB0, EfficientNetB3

from PIL import Image
import datetime
#from efficientnet.preprocessing import center_crop_and_resize
from sklearn.model_selection import train_test_split

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


Inputs

In [9]:
img_arg=Sequential(
    [
     preprocessing.RandomRotation(factor=0.15),
     preprocessing.RandomTranslation(height_factor=0.1, width_factor=0.1),
     preprocessing.RandomFlip(),
     preprocessing.RandomContrast(factor=0.1)

    ]
)

모델 만들기

In [10]:
def build_model(n):
#     inputs=layers.Input(shape=(600,600,3))  #B7을 이용할거니까 가로 세로 600
    width=[224, 240, 260, 300, 380, 456, 528, 600] 

    inputs=layers.Input(shape=(width[n],width[n],3))  #B7을 이용할거니까 가로 세로 600
    
    x=img_arg(inputs)
    
    if n==0:
        model=EfficientNetB0(include_top=False,
                           input_tensor=x,
                           weights='imagenet'
                           )
        
    elif n==1:
        model=EfficientNetB1(include_top=False,
                           input_tensor=x,
                           weights='imagenet'
                           )
        
    elif n==2:
        model=EfficientNetB2(include_top=False,
                           input_tensor=x,
                           weights='imagenet'
                           )
        
    elif n==3:
        model=EfficientNetB3(include_top=False,
                           input_tensor=x,
                           weights='imagenet'
                           )
        
    elif n==4:
        model=EfficientNetB4(include_top=False,
                           input_tensor=x,
                           weights='imagenet'
                           )
        
    elif n==5:
        model=EfficientNetB5(include_top=False,
                           input_tensor=x,
                           weights='imagenet'
                           )
        
    elif n==6:
        model=EfficientNetB6(include_top=False,
                           input_tensor=x,
                           weights='imagenet'
                           )

    elif n==7:
        model=EfficientNetB7(include_top=False,
                           input_tensor=x,
                           weights='imagenet'
                           )
        
        
    
    model.trainable=False

  #build top
    x=layers.GlobalAveragePooling2D(name='avg_pool')(model.output)
    x=layers.BatchNormalization()(x)

    top_dropout_rate=0.2
    x=layers.Dropout(top_dropout_rate, name='top_dropout')(x)

    outputs=layers.Dense(4,
                       activation='softmax',
                       name='pred')(x)
  #compile 
    model=tf.keras.Model(inputs, outputs, name='EfficientNet')
    optimizer=tf.keras.optimizers.Adam()

    model.compile(optimizer=optimizer,
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
    model.save('test', save_format='h5')
    return model

EfficientNetB7(600*600 이미지) 모델을 불러오고 weights는 imagenet weights로 설정. class는 7개로

In [4]:
model=build_model()

C:\Users\user\anaconda3\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [5]:
model.summary()

Model: "EfficientNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 sequential (Sequential)        (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['sequential[0][0]']             
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']   

In [7]:
for layer in model.layers[50:]:
    layer.trainable=True

In [8]:
len(model.layers)

242

클래스 확인


In [12]:
train_path='C:/Users/user/python/FacialClassification/train/resize_600'
test_path='./FacialClassification/test'

In [10]:
classes = os.listdir(train_path)
cls=[]
for i in classes:
    if len(i)<3:
        cls.append(i)
        
cls

['기쁨', '당황', '분노', '슬픔']

데이터 로드

In [11]:
path='C:/Users/user/python/FacialClassification/'

이미지 확인

이미지의 주소는 train_path/test_path, classes[i], files[i][0] 의 형태로 불러올것

# 데이터셋 준비 1안

In [ ]:
# X=[]
# y=[]
# t=0
# for i in range(0,len(files),1):
#   print("=======================================",i)
#   for j in range(len(files[i])):
#     try:
#       a=os.path.join(train_path, classes[i],files[i][j])
#       a=np.array(Image.open(a))
#       # a=center_crop_and_resize(a,600)
#       # a=np.expand_dims(a,0)
#       X.append(a)
#       y.append(classes[i])
#       t+=1
#       print(f'{t}/{b}    {t/b*100}%')
#     except Exception as e:
#       print(e)






# 데이터셋 준비 2안

In [13]:
train_ds=tf.keras.preprocessing.image_dataset_from_directory(train_path,
#                                                              image_size=(600,600),
                                                             image_size=(240,240),
                                                             seed=123)
val_ds=tf.keras.preprocessing.image_dataset_from_directory(test_path,
                                                             image_size=(240,240),
#                                                            image_size=(600,600),
                                                           seed=123)

Found 149743 files belonging to 4 classes.
Found 1944 files belonging to 4 classes.


In [14]:
class_names=train_ds.class_names
print(class_names)

['기쁨', '당황', '분노', '슬픔']


In [18]:
# 모델 저장
save_path = './Model_B0_192/model_{epoch:03d}_{val_accuracy:.4f}.h5'
mckp=ModelCheckpoint(filepath=save_path,   # 저장할 위치
                     monitor='val_accuracy',    # val_accuracy 기준으로다가
                     save_best_only=True,     # 점수가 제일 높은 모델만 저장할거임
                     verbose=1
                     )
early=EarlyStopping(monitor='val_accuracy',
                    patience=30,             # val_accuracy가 5번까지는 안좋아져도 봐줌. 
                    )


In [15]:
dir_name='learning_log'
root_logdir=os.path.join(path, dir_name)
if not os.path.exists(root_logdir):
    os.mkdir(root_logdir)
sub_dir_name=datetime.datetime.now().strftime('%Y%m%d-%H%M%S')

In [16]:
tb_log_dir=os.path.join(root_logdir, sub_dir_name)
tensor_board=tf.keras.callbacks.TensorBoard(log_dir=tb_log_dir)

In [17]:
if not os.path.exists('./Model_B0_192'):
    os.mkdir('./Model_B0_192')

In [19]:
h=model.fit(train_ds,
            validation_data=val_ds,
            epochs=500,
            callbacks=[mckp, early, tensor_board],
            batch_size=32
            )

Epoch 1/500


C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\util\dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


4680/4680 [==============================] - ETA: 0s - loss: 1.2641 - accuracy: 0.4561
Epoch 00001: val_accuracy improved from -inf to 0.50051, saving model to ./Model_B0_192\model_001_0.5005.h5
4680/4680 [==============================] - 512s 106ms/step - loss: 1.2641 - accuracy: 0.4561 - val_loss: 1.3199 - val_accuracy: 0.5005
Epoch 2/500
4679/4680 [============================>.] - ETA: 0s - loss: 1.2136 - accuracy: 0.4734
Epoch 00002: val_accuracy did not improve from 0.50051
4680/4680 [==============================] - 494s 105ms/step - loss: 1.2135 - accuracy: 0.4734 - val_loss: 1.3057 - val_accuracy: 0.4748
Epoch 3/500
4679/4680 [============================>.] - ETA: 0s - loss: 1.1939 - accuracy: 0.4800
Epoch 00003: val_accuracy improved from 0.50051 to 0.51646, saving model to ./Model_B0_192\model_003_0.5165.h5
4680/4680 [==============================] - 491s 105ms/step - loss: 1.1939 - accuracy: 0.4799 - val_loss: 1.3236 - val_accuracy: 0.5165
Epoch 4/500
4679/4680 [=======

In [20]:
# 모델 저장
save_path = './Model_B0_112/model_{epoch:03d}_{val_accuracy:.4f}.h5'
mckp=ModelCheckpoint(filepath=save_path,   # 저장할 위치
                     monitor='val_accuracy',    # val_accuracy 기준으로다가
                     save_best_only=True,     # 점수가 제일 높은 모델만 저장할거임
                     verbose=1
                     )
early=EarlyStopping(monitor='val_accuracy',
                    patience=30,             # val_accuracy가 5번까지는 안좋아져도 봐줌. 
                    )

In [21]:
h=model.fit(train_ds,
            validation_data=val_ds,
            epochs=500,
            callbacks=[mckp, early, tensor_board],
            batch_size=32
            )

Epoch 1/500
4679/4680 [============================>.] - ETA: 0s - loss: 1.1617 - accuracy: 0.5181
Epoch 00001: val_accuracy improved from -inf to 0.56636, saving model to ./Model_B0_112\model_001_0.5664.h5
4680/4680 [==============================] - 528s 112ms/step - loss: 1.1617 - accuracy: 0.5181 - val_loss: 1.2771 - val_accuracy: 0.5664
Epoch 2/500
4679/4680 [============================>.] - ETA: 0s - loss: 1.1622 - accuracy: 0.5170
Epoch 00002: val_accuracy did not improve from 0.56636
4680/4680 [==============================] - 510s 109ms/step - loss: 1.1622 - accuracy: 0.5170 - val_loss: 1.2619 - val_accuracy: 0.5098
Epoch 3/500
4680/4680 [==============================] - ETA: 0s - loss: 1.1610 - accuracy: 0.5187
Epoch 00003: val_accuracy improved from 0.56636 to 0.59516, saving model to ./Model_B0_112\model_003_0.5952.h5
4680/4680 [==============================] - 534s 114ms/step - loss: 1.1610 - accuracy: 0.5187 - val_loss: 1.2760 - val_accuracy: 0.5952
Epoch 4/500
4680/4

Epoch 29/500
4679/4680 [============================>.] - ETA: 0s - loss: 1.1598 - accuracy: 0.5186
Epoch 00029: val_accuracy did not improve from 0.63735
4680/4680 [==============================] - 501s 107ms/step - loss: 1.1598 - accuracy: 0.5186 - val_loss: 1.2594 - val_accuracy: 0.5617
Epoch 30/500
4679/4680 [============================>.] - ETA: 0s - loss: 1.1592 - accuracy: 0.5187
Epoch 00030: val_accuracy did not improve from 0.63735
4680/4680 [==============================] - 501s 107ms/step - loss: 1.1592 - accuracy: 0.5187 - val_loss: 1.2482 - val_accuracy: 0.4408
Epoch 31/500
4679/4680 [============================>.] - ETA: 0s - loss: 1.1620 - accuracy: 0.5181
Epoch 00031: val_accuracy did not improve from 0.63735
4680/4680 [==============================] - 500s 107ms/step - loss: 1.1620 - accuracy: 0.5181 - val_loss: 1.2741 - val_accuracy: 0.5983
Epoch 32/500
4679/4680 [============================>.] - ETA: 0s - loss: 1.1599 - accuracy: 0.5176
Epoch 00032: val_accurac

### B1모델, Trainable layer 절반

In [15]:
model=build_model(1)

In [16]:
save_path = './Model_B1_harf/model_{epoch:03d}_{val_accuracy:.4f}.h5'
mckp=ModelCheckpoint(filepath=save_path,   # 저장할 위치
                     monitor='val_accuracy',    # val_accuracy 기준으로다가
                     save_best_only=True,     # 점수가 제일 높은 모델만 저장할거임
                     verbose=1
                     )
early=EarlyStopping(monitor='val_accuracy',
                    patience=30,             # val_accuracy가 5번까지는 안좋아져도 봐줌. 
                    )

In [17]:
for layer in model.layers[170:]:
    layer.trainable=True

In [18]:
len(model.layers)

344

In [20]:
h1_1=model.fit(train_ds,
            validation_data=val_ds,
            epochs=500,
            callbacks=[mckp, early],
            batch_size=32
            )

Epoch 1/500


C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\util\dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


4680/4680 [==============================] - ETA: 0s - loss: 1.2733 - accuracy: 0.4622
Epoch 00001: val_accuracy improved from -inf to 0.42798, saving model to ./Model_B1_harf\model_001_0.4280.h5
4680/4680 [==============================] - 739s 154ms/step - loss: 1.2733 - accuracy: 0.4622 - val_loss: 1.3511 - val_accuracy: 0.4280
Epoch 2/500
4680/4680 [==============================] - ETA: 0s - loss: 1.2224 - accuracy: 0.4825
Epoch 00002: val_accuracy improved from 0.42798 to 0.57716, saving model to ./Model_B1_harf\model_002_0.5772.h5
4680/4680 [==============================] - 703s 150ms/step - loss: 1.2224 - accuracy: 0.4825 - val_loss: 1.3553 - val_accuracy: 0.5772
Epoch 3/500
4680/4680 [==============================] - ETA: 0s - loss: 1.2045 - accuracy: 0.4963
Epoch 00003: val_accuracy did not improve from 0.57716
4680/4680 [==============================] - 697s 149ms/step - loss: 1.2045 - accuracy: 0.4963 - val_loss: 1.2808 - val_accuracy: 0.5401
Epoch 4/500
4680/4680 [=====

Epoch 29/500
4680/4680 [==============================] - ETA: 0s - loss: 1.1703 - accuracy: 0.5428
Epoch 00029: val_accuracy did not improve from 0.62706
4680/4680 [==============================] - 669s 143ms/step - loss: 1.1703 - accuracy: 0.5428 - val_loss: 1.2666 - val_accuracy: 0.5694
Epoch 30/500
4680/4680 [==============================] - ETA: 0s - loss: 1.1704 - accuracy: 0.5445
Epoch 00030: val_accuracy did not improve from 0.62706
4680/4680 [==============================] - 669s 143ms/step - loss: 1.1704 - accuracy: 0.5445 - val_loss: 1.2742 - val_accuracy: 0.5427
Epoch 31/500
4680/4680 [==============================] - ETA: 0s - loss: 1.1695 - accuracy: 0.5408
Epoch 00031: val_accuracy did not improve from 0.62706
4680/4680 [==============================] - 669s 143ms/step - loss: 1.1695 - accuracy: 0.5408 - val_loss: 1.2563 - val_accuracy: 0.5705
Epoch 32/500
4680/4680 [==============================] - ETA: 0s - loss: 1.1696 - accuracy: 0.5421
Epoch 00032: val_accurac

### B1모델, Trainable Layer 1/4

In [21]:
model=build_model(1)

In [22]:
save_path = './Model_B1_quarter/model_{epoch:03d}_{val_accuracy:.4f}.h5'
mckp=ModelCheckpoint(filepath=save_path,   # 저장할 위치
                     monitor='val_accuracy',    # val_accuracy 기준으로다가
                     save_best_only=True,     # 점수가 제일 높은 모델만 저장할거임
                     verbose=1
                     )
early=EarlyStopping(monitor='val_accuracy',
                    patience=30,             # val_accuracy가 5번까지는 안좋아져도 봐줌. 
                    )

In [23]:
for layer in model.layers[len(model.layers)/4:]:
    layer.trainable=True

TypeError: slice indices must be integers or None or have an __index__ method

In [ ]:
h1_2=model.fit(train_ds,
            validation_data=val_ds,
            epochs=500,
            callbacks=[mckp, early],
            batch_size=32
            )

### B1모델, Trainable Layer 3/4

In [ ]:
model=build_model(1)

In [ ]:
save_path = './Model_B1_3quarter/model_{epoch:03d}_{val_accuracy:.4f}.h5'
mckp=ModelCheckpoint(filepath=save_path,   # 저장할 위치
                     monitor='val_accuracy',    # val_accuracy 기준으로다가
                     save_best_only=True,     # 점수가 제일 높은 모델만 저장할거임
                     verbose=1
                     )
early=EarlyStopping(monitor='val_accuracy',
                    patience=30,             # val_accuracy가 5번까지는 안좋아져도 봐줌. 
                    )

In [ ]:
for layer in model.layers[3*len(model.layers)/4:]:
    layer.trainable=True

In [ ]:
h1_3=model.fit(train_ds,
            validation_data=val_ds,
            epochs=500,
            callbacks=[mckp, early],
            batch_size=32
            )